In [8]:
from netCDF4 import Dataset
import rasterio

from wrf import (to_np, getvar, smooth2d, get_cartopy, cartopy_xlim,
                 cartopy_ylim, latlon_coords, interplevel)
from wrf import ll_to_xy

import numpy as np
import pandas as pd
from PIL import Image
from PIL.Image import Resampling
import PIL
# from osgeo import gdal
import glob
import datetime

from tqdm.notebook import tqdm
# We'll be using widgets in the notebook
import ipywidgets as widgets 
from IPython.display import display

# transfering WRF output to .NPY format

In [ ]:
case_path = 'path to wrf output'

date = '20210820'
var_names = ['T2','rh2','wspd10','wdir10', 'SWDOWN', 'GLW', 'PSFC']

paths = glob.glob(case_path+'\%s'%date+'\wrfout_d04*')

from tqdm import tqdm
for var_name in var_names:
    i = 0
    for path in tqdm(paths, desc='var_name'):
        UTC0 = path[-19:]
        UTC8 = pd.to_datetime(UTC0, format='%Y-%m-%d_%H_%M_%S') + datetime.timedelta(hours=8)
        # print(LTC, ' ')
        ncfile = Dataset(path)

        if var_name in ['T2','TSK']:
            field = to_np(getvar(ncfile, var_name)-273.15)
        elif var_name=='PSFC':
            field = to_np(getvar(ncfile, var_name)/1000)  # Pa to KPa
        else:
            field = to_np(getvar(ncfile, var_name))
        if i ==0:
            ary = field
        else:
            ary = np.dstack((ary, field))               

        i+=1

    ncfile.close()
    np.save(case_path + r'\%s_'%var_name+date, np.array(ary.transpose(2,0,1)))

## initialization

In [7]:
def umep_met_init():
    columns = ['#iy','id','it','imin','Q*','QH','QE','Qs','Qf',
               'Wind','RH','Td','press','rain','Kdn','snow',
               'ldown','fcld','wuh','xsmd','lai_hr','Kdiff','Kdir','Wd']
    # time index
    ids = [232]*16 + [233]*24 + [234]*24 + [235]*9
    it = list(range(8,24))+list(range(24))+list(range(24))+list(range(9))
    iy, imin = [2021]*73, [0]*73
    rain = [0]*73

    umep_met = pd.DataFrame([], columns = columns)
    umep_met['#iy'], umep_met.id, umep_met.it, umep_met.imin = iy, ids, it, imin
    umep_met.rain = rain
    umep_met = umep_met.fillna(-999)
    
    return umep_met

In [ ]:
case_path = 'path to wrf output'

var_names = ['T2','rh2','wspd10','wdir10', 'SWDOWN', 'GLW', 'PSFC']
var_dict = {'wspd10':'Wind','rh2':'RH','T2':'Td','PSFC':'press','SWDOWN':'Kdn','GLW':'ldown','wdir10':'Wd'}

umep_met = umep_met_init()
npy_path = case_path + r'\%s_%s.npy'%('SWDOWN', '20210820')
SWDOWN = np.load(npy_path).mean(axis=1).mean(axis=1)
npy_path = case_path + r'\%s_%s.npy'%('GLW', '20210820')
GLW = np.load(npy_path).mean(axis=1).mean(axis=1)

umep_met[var_dict['SWDOWN']] = SWDOWN
umep_met[var_dict['GLW']] = GLW
umep_met[var_dict['GLW']]

# using mean values for radiation to remove cloud effect

In [ ]:
# var_names = ['T2','rh2','wspd10','wdir10', 'SWDOWN', 'GLW', 'PSFC']
var_names = ['wspd10','rh2','T2','PSFC','wdir10']

var_dict = {'wspd10':'Wind','rh2':'RH','T2':'Td','PSFC':'press','SWDOWN':'Kdn','GLW':'ldown','wdir10':'Wd'}
date='20210820'
case_path = r'E:\p5_UWGs\CASES\20210820\case2\REAL'
output_path = r'E:\p5_UWGs\QGIS proj\UMEP data\MET\wrf2umep'

# using mean values for radiation to remove cloud effect
npy_path = case_path + r'\%s_%s.npy'%('SWDOWN', '20210820')
SWDOWN = np.load(npy_path).mean(axis=1).mean(axis=1)
npy_path = case_path + r'\%s_%s.npy'%('GLW', '20210820')
GLW = np.load(npy_path).mean(axis=1).mean(axis=1)

for nlon in range(120):
    for nlat in range(120):
        umep_met = umep_met_init()
        umep_met[var_dict['SWDOWN']] = SWDOWN
        umep_met[var_dict['GLW']] = GLW
        
        for var_name in var_names:   
            npy_path = case_path + r'\%s_%s.npy'%(var_name, date)
            met_data = np.load(npy_path)
            umep_met[var_dict[var_name]] = met_data[:,nlat,nlon]
            
        umep_met.to_csv(output_path + r'\wrf2umep_%.2d_%.2d.txt'%(nlon, nlat), index=None, sep=' ')